<a href="https://colab.research.google.com/github/Nuccy90/Master_thesis/blob/master/ELMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install allennlp

    100% |████████████████████████████████| 5.6MB 7.3MB/s 
    100% |████████████████████████████████| 122kB 30.4MB/s 
    100% |████████████████████████████████| 1.5MB 15.7MB/s 
    100% |████████████████████████████████| 245kB 31.5MB/s 
    100% |████████████████████████████████| 51kB 24.6MB/s 
    100% |████████████████████████████████| 51kB 24.3MB/s 
    100% |████████████████████████████████| 245kB 26.7MB/s 
    100% |████████████████████████████████| 552kB 28.5MB/s 
    100% |████████████████████████████████| 133kB 32.4MB/s 
    100% |████████████████████████████████| 51kB 20.5MB/s 
    100% |████████████████████████████████| 143kB 29.7MB/s 
    100% |████████████████████████████████| 61kB 23.0MB/s 
    100% |████████████████████████████████| 2.3MB 15.4MB/s 
    100% |████████████████████████████████| 9.7MB 1.1MB/s 
    100% |████████████████████████████████| 92kB 27.4MB/s 
    100% |████████████████████████████████| 204kB 31.0MB/s 
    100% |████████████████████████████████| 102

In [0]:
import sklearn
import pandas as pd
import numpy as np
import re
import itertools
import json
import nltk
from allennlp.commands.elmo import ElmoEmbedder
from tensorflow import keras
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.linear_model import LogisticRegression
from datetime import datetime
from google.colab import drive
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping, ModelCheckpoint

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


Using TensorFlow backend.


In [0]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
def read_data(name):
    
    df = pd.read_csv(name)
    df = df.dropna(subset=['Text'])
    df['Title'] = df['Title'].fillna('')
    df = df[df["Text"] != "  "]
    df = df[df['Title'] + df['Text'] != '']
    
    return df

In [0]:
def cleanText(text):
    
    #sw = stopwords.words('english')[35:]
    text = re.sub(r'\|\|\|', r' ', text)
    text = re.sub(r'http\S+', r'<URL>', text)
    text = re.sub(r'\d{6,}', r'<NUM>', text)
    
    return text

In [0]:
def create_elmo_embeddings(document, elmo):
    
    sentences = sent_tokenize(document)
    embeddings = []
    tokens = [word_tokenize(sentence) for sentence in sentences]
    
    #here you need to decide if you want to cut to 100 words
    #before you do the embeddings
    
    for elmo_embedding in elmo.embed_sentences(tokens):  
        # Average the 3 layers returned from ELMo
        avg_elmo_embedding = np.average(elmo_embedding, axis=0)
             
        embeddings.extend(avg_elmo_embedding)
        
            
    return embeddings


In [0]:
# get vectors

def get_vectors(docs, elmo):

    X = []

    for doc in docs:
    
        doc_embeddings = create_elmo_embeddings(doc, elmo)
        X.append(doc_embeddings)
        
    return X


In [0]:
def sort_by_date(df):

    di = {}        
    for index, row in df.iterrows():

        sub_id = row["Subject"]
        diagnosis = row["Diagnosis"]
        if sub_id in di:
            di[sub_id][1].append(row["Text"])
            di[sub_id][2].append(row["Date"])
        else:
            di[sub_id] = [diagnosis,[row["Text"]], [row["Date"]]]

    for key in di:
        list_of_datetimes = [datetime.strptime(x, ' %Y-%m-%d %H:%M:%S ') for x in di[key][2]]

        lists = sorted(zip(*[list_of_datetimes, di[key][1]]))
        dates, texts = list(zip(*lists))

        di[key][1] = texts
        di[key][2] = dates
        
    return di

In [0]:
def predict_texts(di, model):
    print("Starting prediction...")
    pred_dict = {}
    
    elmo = ElmoEmbedder(cuda_device = 0)
    
    def predict_scores(texts, model = model, elmo = elmo):
    
        #add score to dictionary
        x_texts = get_vectors(texts, elmo)
        x_texts = sequence.pad_sequences(x_texts, maxlen=100)
        if len(texts) == 1:
            print(x_texts)
            scores = model.predict(x_texts)[0]
        else:
            scores = model.predict(x_texts)
            
        return scores
    
    for sub_id in di:
        texts = di[sub_id][1]
        pred_dict[sub_id] = predict_scores(texts)
    
    print("Prediction done!")
    return pred_dict

In [0]:
def create_train_vectors(pred_dict, di):
    
    x_train = []
    y_train = []

    for i in range(1,2000):
        for sub_id in pred_dict:
        
            if i >= len(pred_dict[sub_id]):
                pass
            else:
                seen = pred_dict[sub_id][:i]
                avg = np.mean(seen)
                sd = np.std(seen)
                top_n = int(round((20*i)/100))
                topn_avg = np.mean(np.sort(seen)[top_n:])
                bottomn_avg = np.mean(np.sort(seen)[:top_n+1])
                diff = topn_avg - bottomn_avg
                n_texts = (i-1)/(1999-1)
            
                x = np.array([n_texts,avg,sd,topn_avg,diff])
                x_train.append(x)
                y_train.append(di[sub_id][0])
                
    return np.array(x_train), np.array(y_train)

In [0]:
def predict_test_vectors(pred_dict, di, clf2):
    
    verdict_dict = {}
    
    for i in range(1,2000):
        for sub_id in pred_dict:
        
            if i >= len(pred_dict[sub_id]):
                verdict_dict[sub_id].append(verdict_dict[sub_id][-1])
            else:
                seen = np.array(pred_dict[sub_id][:i])
                avg = np.mean(seen)
                sd = np.std(seen)
                top_n = int(round((20*i)/100))
                topn_avg = np.mean(np.sort(seen)[top_n:])
                bottomn_avg = np.mean(np.sort(seen)[:top_n+1])
                diff = topn_avg - bottomn_avg
                n_texts = (i-1)/(1999-1)

                x = np.array([n_texts,avg,sd,topn_avg,diff])
                x = x.reshape(1,-1)
                verdict = clf2.predict(x)

                if sub_id in verdict_dict:
                    verdict_dict[sub_id].append(verdict)
                else:
                    verdict_dict[sub_id] = [verdict]
                
    return verdict_dict

In [0]:
def evaluate(verdict_dict, o):
    
    # create dataframe to hold the data necessary for the final calculations
    cols = ["subject", "true_risk", "risk_decision", "delay", "erde"]

    df_final = pd.DataFrame(index = range(94),columns = cols)

    count = 0
    with open("/content/drive/My Drive/risk_test_users.txt", 'r') as f:
        for line in f.readlines():
            df_final.iloc[count]['subject'] = line.split('\t')[0].strip()
            df_final.iloc[count]['true_risk'] = float(line.split('\t')[1].strip())
            count += 1


    # here i put the risk_decision and delay in the dataframe  
    for key in verdict_dict:
        sub_row = df_final.index[df_final['subject'] == key].tolist()[0]

        df_final.iloc[sub_row,2] = verdict_dict[key][-1]

        if (df_final.iloc[sub_row,2] == 1) & (df_final.iloc[sub_row,1] == 1):
            df_final.iloc[sub_row,3] = verdict_dict[key].index(1)

    #extract the data
    risk_d = df_final['risk_decision']
    t_risk = df_final['true_risk']
    k = df_final['delay']
    erde = df_final['erde']

    # Count of how many true positives there are
    true_pos = len(df_final[t_risk==1])

    # Count of how many positive cases the system decided there were
    pos_decisions = len(df_final[risk_d==1])

    # Count of how many of them are actually true positive cases
    pos_hits = len(df_final[(t_risk==1) & (risk_d==1)])

    # Total count of users
    total_users = len(df_final)

    # ERDE calculations
    for i in range(total_users):
        if(risk_d[i] == 1 and t_risk[i] == 0):
            erde.iloc[i] = float(true_pos)/total_users
        elif(risk_d[i] == 0 and t_risk[i] == 1):
            erde.iloc[i] = 1.0
        elif(risk_d[i] == 1 and t_risk[i] == 1):
            erde.iloc[i] = 1.0 - (1.0/(1.0+np.exp(k[i]-o)))
        elif(risk_d[i] == 0 and t_risk[i] == 0):
            erde.iloc[i] = 0.0

    # Calculation of F1, Precision, Recall and global ERDE
    precision = float(pos_hits)/pos_decisions
    recall = float(pos_hits)/true_pos
    F1 = 2 * (precision * recall) / (precision + recall)
    erde_global = erde.mean() * 100

    #indiv_erde = df_final.iloc[:,['subject','erde']]
    #print (indiv_erde.to_string())
    print ('Global ERDE (with o = %d): %.2f' % (o, erde_global), '%')
    print ('F1: %.2f' % F1)
    print ('Precision: %.2f' % precision)
    print ('Recall: %.2f' % recall)
    return df_final

In [0]:
# read data, clean up texts

df_train = read_data('/content/drive/My Drive/training.csv')
df_test = read_data('/content/drive/My Drive/test.csv')

#df_train = df_train.sample(frac=0.0005)
#df_test = df_test.sample(frac=0.001)

docs_train = df_train['Title'] + df_train['Text']
Y_train = df_train["Diagnosis"].values

docs_test = df_test['Title'] + df_test['Text']
Y_test = df_test["Diagnosis"].values

docs_train = docs_train.apply(cleanText)
docs_test = docs_test.apply(cleanText)

In [0]:
elmo = ElmoEmbedder()

X_train = get_vectors(docs_train, elmo)

In [0]:
max_text_length = 100
X_train = sequence.pad_sequences(X_train, maxlen=max_text_length)

In [0]:
# create Keras classifier

embedding_dim = 1024

model = Sequential()
model.add(LSTM(100, input_shape=(100, 1024)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100)               450000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 450,101
Trainable params: 450,101
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
filepath="/content/drive/My Drive/elmo_toy.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint, EarlyStopping(monitor='val_loss',min_delta=0.0001)]
model.fit(X_train, Y_train, validation_split=0.2, callbacks=callbacks_list, epochs=5, batch_size=32)

Train on 66 samples, validate on 17 samples
Epoch 1/5
66/66 [==============================] - 2s 30ms/step - loss: 0.6482 - acc: 0.6515 - val_loss: 0.4148 - val_acc: 0.9412

Epoch 00001: val_acc improved from -inf to 0.94118, saving model to /content/drive/My Drive/elmo_toy.hdf5
Epoch 2/5
66/66 [==============================] - 1s 13ms/step - loss: 0.4480 - acc: 0.8485 - val_loss: 0.2691 - val_acc: 0.9412

Epoch 00002: val_acc did not improve from 0.94118
Epoch 3/5
66/66 [==============================] - 1s 13ms/step - loss: 0.4047 - acc: 0.8485 - val_loss: 0.2358 - val_acc: 0.9412

Epoch 00003: val_acc did not improve from 0.94118
Epoch 4/5
66/66 [==============================] - 1s 13ms/step - loss: 0.3812 - acc: 0.8485 - val_loss: 0.2478 - val_acc: 0.9412

Epoch 00004: val_acc did not improve from 0.94118


In [0]:
model = keras.models.load_model('/content/drive/My Drive/Models/elmo_model.hdf5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [0]:
# sort texts by date and make predictions for both training and test set
df_train['Text'] = df_train['Text'].apply(cleanText)
df_test['Text'] = df_test['Text'].apply(cleanText)

dict_train = sort_by_date(df_train)
dict_test = sort_by_date(df_test)

In [0]:
pred_dict_train = predict_texts(dict_train, model)
pred_dict_test = predict_texts(dict_test, model)

In [0]:
texts = pd.Series(["This is document number one. Not so interesting."])
elmo = ElmoEmbedder(cuda_device = 0)
x_texts = get_vectors(texts, elmo)
x_texts = sequence.pad_sequences(x_texts, maxlen=100)
if len(texts) == 1:
    scores = model.predict(x_texts)[0]
else:
    scores = model.predict(x_texts)

In [0]:
scores

array([0.08282965], dtype=float32)

In [0]:
# get the feature vectors and train the person classifier

x_train, y_train = create_train_vectors(pred_dict_train, dict_train)

clf2 = LogisticRegression(max_iter=2000, class_weight = 'balanced',solver='saga')
clf2.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=2000,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='saga', tol=0.0001, verbose=0, warm_start=False)

In [0]:
x_test, y_test = create_train_vectors(pred_dict_test, dict_test)
clf2.score(x_test, y_test)

0.6428571428571429

In [0]:
# predict test set and evaluate

verdict_dict = predict_test_vectors(pred_dict_test, dict_test, clf2)
df_final = evaluate(verdict_dict, 50)

In [0]:
df_final